In [1]:
# ---------- SECOND LEVEL ANALYSIS ------------
# ---------------- Default HRF ----------------

# Imports
import os
import glob
import nilearn 
import itertools
import numpy as np 
import pandas as pd
import nibabel as nib

from numpy import savetxt
import matplotlib.pyplot as plt
from nilearn import plotting
from nilearn.glm import threshold_stats_img
from nilearn.image import get_data, math_img
from nilearn import plotting, image, datasets
from nilearn.plotting import plot_design_matrix
from nilearn.datasets import fetch_localizer_contrasts
from nilearn.glm.second_level import SecondLevelModel
from nilearn.glm.second_level import make_second_level_design_matrix
#from nilearn.glm.second_level import non_parametric_inference
#from scipy.stats import norm

# Init variables
init_folder='/home/traaffneu/margal/code/multirat_se/script/'
analysis_folder='/project/4180000.19/multirat_stim/scratch/rabies_test/first_level/'
z_scores_path = "/project/4180000.19/multirat_stim/scratch/rabies_test/first_level/z_score/"
beta_path = "/project/4180000.19/multirat_stim/scratch/rabies_test/first_level/beta_estimates/"

# Data path
template_path ='/groupshare/traaffneu/preclinimg/templates/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Template.nii'
metadata_path = '/home/traaffneu/margal/code/multirat_se/script/table/metadata_stand.tsv'

# Output path
output_dir = '/project/4180000.19/multirat_stim/scratch/rabies_test/second_level/'
glover_dir =  os.path.join(output_dir, 'glover/')
spm_dir = os.path.join(output_dir, 'spm/')

glover_image_dir = os.path.join(glover_dir, 'image/')
glover_beta_dir = os.path.join(glover_dir, 'beta_estimates/')
glover_z_score_dir = os.path.join(glover_dir, 'z_score/')
glover_p_value_dir = os.path.join(glover_dir, 'p_value/')
glover_clusters_dir = os.path.join(glover_dir, 'clusters/')

spm_image_dir = os.path.join(spm_dir, 'image/')
spm_beta_dir = os.path.join(spm_dir, 'beta_estimates/')
spm_z_score_dir = os.path.join(spm_dir, 'z_score/')
spm_p_value_dir = os.path.join(spm_dir, 'p_value/')
spm_clusters_dir = os.path.join(spm_dir, 'clusters/')

if not os.path.exists(output_dir): os.makedirs(output_dir)
if not os.path.exists(glover_dir): os.makedirs(glover_dir)
if not os.path.exists(spm_dir): os.makedirs(spm_dir)

if not os.path.exists(glover_image_dir): os.makedirs(glover_image_dir)
if not os.path.exists(glover_beta_dir): os.makedirs(glover_beta_dir)
if not os.path.exists(glover_z_score_dir): os.makedirs(glover_z_score_dir)
if not os.path.exists(glover_p_value_dir): os.makedirs(glover_p_value_dir)
if not os.path.exists(glover_clusters_dir): os.makedirs(glover_clusters_dir)
if not os.path.exists(spm_image_dir): os.makedirs(spm_image_dir)
if not os.path.exists(spm_beta_dir): os.makedirs(spm_beta_dir)
if not os.path.exists(spm_z_score_dir): os.makedirs(spm_z_score_dir)
if not os.path.exists(spm_p_value_dir): os.makedirs(spm_p_value_dir)
if not os.path.exists(spm_clusters_dir): os.makedirs(spm_clusters_dir)

df = pd.read_csv(metadata_path, sep='\t')
df = df.loc[(df['exclude'] != 'yes')]


In [2]:
# --- Input data ---

""" Define hrf model used for the first analysis
    Options : 
    ---------
    spm
    glover
"""
hrf_function = 'glover'
#hrf_function = 'spm'

In [3]:
# ---------- SECOND LEVEL ANALYSIS ------------
# ---------------- Default HRF ----------------

print('First level analysis used:', hrf_function)

for index in range(5, 6):
    
    ID = "0"+str(2000+index)
    print("ID:", ID)
    
    #if ID == "02010":
     #   continue
 
    # -- Load data --
    if hrf_function == 'glover':
        dataset = glob.glob(analysis_folder+'glover/beta_estimates/beta_sub-{}??_ses-1.nii.gz'.format(ID))
    elif hrf_function == 'spm':
        dataset = glob.glob(analysis_folder+'spm/beta_estimates/beta_sub-{}??_ses-1.nii.gz'.format(ID))
    else:
        print('No file to be saved. Check that hrf_function is well defined.')
    
    n_subject = len(dataset)                                                           
    print("number of subjects:", n_subject)

    second_level_input = dataset
    design_matrix = pd.DataFrame([1] * len(second_level_input), 
                                columns=["intercept"])


    # --- Specify the model and fit it --- 
    second_level_model = SecondLevelModel(smoothing_fwhm=0.1,
                                          minimize_memory=False)        #if want to get residuals?

    second_level_model = second_level_model.fit(second_level_input,
                                                design_matrix=design_matrix)


    # --- Estimate the contrast --- 
    second_stat_map = second_level_model.compute_contrast(second_level_contrast='intercept', output_type='all') 

    # nib.save(second_stat_map['z_score'], z_score_dir+'z_score_dataset-{}.nii.gz'.format(ID))          #save z_score map
    # nib.save(second_stat_map['effect_size'], beta_dir+'beta_dataset-{}.nii.gz'.format(ID))            #save the beta estimates
    # nib.save(second_stat_map['p_value'], p_value_dir+'p_value_dataset-{}.nii.gz'.format(ID))          #save the p_value

    p_val = 0.05
    plot_stat = plotting.plot_stat_map(second_stat_map['z_score'],
                                        bg_img = template_path,
                                        threshold = 1,              #threshold p=p_val=0.05 uncorrected -> reduces false negative levels
                                        cut_coords= (0 ,0, 5.5),        
                                        display_mode='ortho',
                                        draw_cross=True,
                                        colorbar=True,
                                        vmax = 5, 
                                        title="Dataset {}".format(ID))
    
    #-- Save outputs --  
    if hrf_function == 'glover':
        nib.save(second_stat_map['z_score'], glover_z_score_dir+'z_score_dataset-{}.nii.gz'.format(ID))          #save z_score map
        nib.save(second_stat_map['effect_size'], glover_beta_dir+'beta_dataset-{}.nii.gz'.format(ID))            #save the beta estimates
        nib.save(second_stat_map['p_value'], glover_p_value_dir+'p_value_dataset-{}.nii.gz'.format(ID))          #save the p_value
        plt.savefig(glover_image_dir+'stat_map_dataset-{}_z_score.png'.format(ID)) 

    elif hrf_function == 'spm':
        #nib.save(second_stat_map['z_score'], spm_z_score_dir+'z_score_dataset-{}.nii.gz'.format(ID))          #save z_score map
        #nib.save(second_stat_map['effect_size'], spm_beta_dir+'beta_dataset-{}.nii.gz'.format(ID))            #save the beta estimates
        #nib.save(second_stat_map['p_value'], spm_p_value_dir+'p_value_dataset-{}.nii.gz'.format(ID))          #save the p_value
        plt.savefig(spm_image_dir+'stat_map_dataset-{}_z_score.png'.format(ID)) 

    else:
        print('No file to be saved. Check that hrf_function is well defined.')
        

First level analysis used: glover
ID: 02005
number of subjects: 10


In [10]:
    #-- Extract clusters -- https://nilearn.github.io/dev/auto_examples/04_glm_first_level/plot_predictions_residuals.html#sphx-glr-auto-examples-04-glm-first-level-plot-predictions-residuals-py

    from nilearn.reporting import get_clusters_table
    from nilearn.maskers import NiftiSpheresMasker

    colorbar = plot_stat._cbar                      # get the statistical scale from the plot
    vmin, vmax = colorbar.mappable.get_clim()
    set_tresh = vmax / 2                            # get the stat_threshold for cluster, depending on the dataset
    print("cluster treshold: ", set_tresh)
    
    table = get_clusters_table(second_stat_map['z_score'],
                            stat_threshold=set_tresh,
                            cluster_threshold=40)


    table.set_index("Cluster ID", drop=True)
    print('Cluster table: ok')

    coords = table.loc[range(0, len(table)), ['X', 'Y', 'Z']].values                     # get the clusters' x, y, and z coordinates (if want the 4 largest, range(0, 4)
    coords = coords.reshape(-1, 3)                                                       # reshape in 3D
    print("(nb clusters, dimension):", coords.shape)                                     # output = (number_clusters, 3)
    
    timeseries_masker = NiftiSpheresMasker(coords, radius=1)                             # extracts time series data from a set of spherical regions of interest (ROIs) in a 3D fMRI image
    
    fmri_img = image.concat_imgs(dataset)

    #real_timeseries = timeseries_masker.fit_transform(fmri_img)                          # applies the masker to an fMRI image, outputs 2D numpy array. Returns signal for each sphere, shape: (number of scans, number of spheres)
    #savetxt(clusters_dir+'cluster_timeseries_dataset-0{}.csv'.format(ID), real_timeseries, delimiter=',')       #save as .csv file



NameError: name 'plot_stat' is not defined

In [6]:
    #-- Extract clusters -- 
    # https://nilearn.github.io/dev/auto_examples/04_glm_first_level/plot_predictions_residuals.html#sphx-glr-auto-examples-04-glm-first-level-plot-predictions-residuals-py

    from nilearn.reporting import get_clusters_table
    from nilearn.maskers import NiftiSpheresMasker

    colorbar = plot_stat._cbar                      # get the statistical scale from the plot
    vmin, vmax = colorbar.mappable.get_clim()
    set_tresh = vmax / 2                            # get the stat_threshold for cluster, depending on the dataset
    print("cluster treshold: ", set_tresh)
    
    table = get_clusters_table(second_stat_map['z_score'],
                            stat_threshold=set_tresh,
                            cluster_threshold=40)


    table.set_index("Cluster ID", drop=True)
    print('Cluster table: ok')

    coords = table.loc[range(0, len(table)), ['X', 'Y', 'Z']].values                     # get the clusters' x, y, and z coordinates (if want the 4 largest, range(0, 4)
    coords = coords.reshape(-1, 3)                                                       # reshape in 3D
    masker = NiftiSpheresMasker(coords, radius=1)                                        # extracts time series data from a set of spherical regions of interest (ROIs) in a 3D fMRI image

    print("(nb clusters, dimension):", coords.shape)                                     # output = (number_clusters, 3)
    
    fmri_img = image.concat_imgs(dataset)                                                 
    real_timeseries = masker.fit_transform(fmri_img)                                     # applies the masker to an fMRI image, outputs 2D numpy array
    
    savetxt(clusters_dir+'cluster_dataset-0{}.csv'.format(ID), real_timeseries, delimiter=',')       #save as .csv file

#-----
    # predicted_timeseries = masker.fit_transform(second_level_model.predicted[0])
    #resid_cluster = masker.fit_transform(second_level_model.residuals[0])
    
    
    

cluster treshold:  1.4046857357025146
Cluster table: ok
(nb clusters, dimension): (3, 3)


In [7]:
print(coords.shape)
print(table)

(3, 3)
   Cluster ID         X         Y         Z  Peak Stat  Cluster Size (mm3)
0           1  5.860001 -1.309999 -2.265000   2.736944                   2
1           2  5.260001  0.190001  7.635001   2.334762                   1
2           3 -7.039999 -1.309999  7.635001   2.032514                   1
